In [1]:
import os
from datetime import datetime

import pandas as pd
from pandas import DataFrame, Series
import numpy as np

from scipy.sparse import lil_matrix

# Data Loading

In [322]:
%%time
location_data=DataFrame()
loaded_files = []
for filename in os.listdir("data/locations"):
    if filename.endswith(".csv"):
        loaded_files.append(pd.read_csv("data/locations/"+filename))
location_data = pd.concat(loaded_files)

detail_data=DataFrame()
loaded_files = []
for filename in os.listdir("data/details"):
    if filename.endswith(".csv"):
        loaded_files.append(pd.read_csv("data/details/"+filename))
detail_data = pd.concat(loaded_files)
detail_data.reset_index(drop=True,inplace=True)

<string>:2: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 31.8 s


In [323]:
location_data['YEAR'] = (location_data.YEARMONTH-location_data.YEARMONTH%100)//100
location_data['MONTH'] = location_data.YEARMONTH%100
location_data = location_data.drop(['YEARMONTH', "LAT2", "LON2"], axis=1)

In [324]:
location_data.head()

,EPISODE_ID,EVENT_ID,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,YEAR,MONTH
0,2030059,5548852,1,NaN,NaN,LANGLEY,34.32,-93.83,1996,3
1,2030060,5548853,1,2.0,S,YELLVILLE,36.20,-92.68,1996,3
2,1002564,5548854,1,NaN,NaN,COTTER,36.27,-92.53,1996,3
3,2030061,5548855,1,NaN,NaN,COTTER,36.27,-92.53,1996,3
4,2030062,5548856,1,5.0,WSW,MOUNTAIN HOME,36.30,-92.47,1996,3


In [325]:
detail_data.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40.0,...,0.0,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48.0,...,0.0,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
3,195007,5,1830,195007,5,1830,NaN,10104928,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
4,195007,24,1440,195007,24,1440,NaN,10104929,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB


# Convert the Time related strings to a DateTime column

In [326]:
%%time
# Pad the "DAY" columns to length 2 so the date parser can do it's thing
detail_data.BEGIN_DAY = detail_data.BEGIN_DAY.map(str).str.pad(2,fillchar='0')
detail_data.END_DAY = detail_data.END_DAY.map(str).str.pad(2,fillchar='0')
# ditto with the Time, except we're padding to the right this time
detail_data.BEGIN_TIME = detail_data.BEGIN_TIME.map(str).str.pad(4,side="right",fillchar='0')
detail_data.END_TIME = detail_data.END_TIME.map(str).str.pad(4,side="right",fillchar='0')
# create a new column by concating the three date/time related columns and convert the result to a datetime
detail_data['BEGIN_DATE']=detail_data.BEGIN_YEARMONTH.map(str)+" "+detail_data.BEGIN_DAY.map(str)+" "+detail_data.BEGIN_TIME.map(str)
detail_data.BEGIN_DATE=pd.to_datetime(detail_data.BEGIN_DATE, format='%Y%m %d %H%M', errors='coerce')

detail_data['END_DATE']=detail_data.END_YEARMONTH.map(str)+" "+detail_data.END_DAY.map(str)+" "+detail_data.END_TIME.map(str)
detail_data.END_DATE=pd.to_datetime(detail_data.END_DATE, format='%Y%m %d %H%M', errors='coerce')

# drop the old columns
detail_data = detail_data.drop(['BEGIN_YEARMONTH', "END_YEARMONTH",'BEGIN_DAY', "END_DAY", "BEGIN_TIME", "END_TIME"], axis=1)
detail_data = detail_data.drop(['YEAR', "MONTH_NAME",'END_DATE_TIME', "BEGIN_DATE_TIME"], axis=1)

# Columns I don't think I need at the momment
detail_data = detail_data.drop(['WFO', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION','EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'], axis=1)

Wall time: 29.6 s


# Correcting input errors
There's some input errors with the Lat/Long coordinates - decimal place is just shifted, so dividing those by ten gives us the correct value (Affects 2080 rows)

In [327]:
detail_data.loc[detail_data.BEGIN_LON < -180, 'BEGIN_LON']=detail_data.loc[detail_data.BEGIN_LON < -180, 'BEGIN_LON']/10
detail_data.loc[detail_data.END_LON < -180, 'END_LON']=detail_data.loc[detail_data.END_LON < -180, 'END_LON']/10

Fix inconsistencies in event type tagging

In [328]:
detail_data.loc[detail_data.EVENT_TYPE.isin(['HAIL FLOODING', 'HAIL/ICY ROADS']), 'EVENT_TYPE'] = 'Hail'
detail_data.loc[detail_data.EVENT_TYPE.isin(['Heavy Wind']), 'EVENT_TYPE'] = 'High Wind'
detail_data.loc[detail_data.EVENT_TYPE.isin(['High Snow']), 'EVENT_TYPE'] = 'Heavy Snow'
detail_data.loc[detail_data.EVENT_TYPE.isin(['High Snow']), 'EVENT_TYPE'] = 'Heavy Snow'
detail_data.loc[detail_data.EVENT_TYPE.isin(
    [
        'TORNADOES, TSTM WIND, HAIL',
        'THUNDERSTORM WINDS/FLOODING',
        'THUNDERSTORM WINDS/FLASH FLOOD',
        'THUNDERSTORM WINDS LIGHTNING',
        'THUNDERSTORM WIND/ TREES',
        'THUNDERSTORM WIND/ TREE',
        'THUNDERSTORM WINDS FUNNEL CLOU',
        'THUNDERSTORM WINDS/HEAVY RAIN',
        'THUNDERSTORM WINDS HEAVY RAIN',
        'THUNDERSTORM WINDS/ FLOOD'
    ]
), 'EVENT_TYPE'] = 'Thunderstorm Wind'
detail_data.loc[detail_data.EVENT_TYPE.isin(['TORNADO/WATERSPOUT']), 'EVENT_TYPE'] = 'Tornado'
detail_data.loc[detail_data.EVENT_TYPE.isin(['Landslide']), 'EVENT_TYPE'] = 'Debris Flow'
detail_data.loc[detail_data.EVENT_TYPE.isin(['Volcanic Ashfall']), 'EVENT_TYPE'] = 'Volcanic Ash'

Drop 14 rows tagged "Northern Lights" and a single row tagged 'OTHER'.

In [329]:
detail_data.drop(detail_data[detail_data.EVENT_TYPE.isin(['Northern Lights', 'OTHER'])].index, inplace=True)

# Create Categorical Index

Create two categories - one based off of the event type and another based on my not very scientific classification

In [330]:
detail_data.EVENT_TYPE = detail_data.EVENT_TYPE.astype("category")
detail_data['EVENT_CODE'] = detail_data.EVENT_TYPE.cat.codes

In [331]:
detail_data['META_TYPE'] = None
detail_data.loc[detail_data.EVENT_TYPE.isin(
    [
        'Avalanche',
        'Blizzard',
        'Cold/Wind Chill',
        'Extreme Cold/Wind Chill',
        'Freezing Fog',
        'Frost/Freeze',
        'Hail',
        'Heavy Snow',
        'Ice Storm',
        'Lake-Effect Snow',
        'Marine Hail',
        'Sleet',
        'Winter Storm',
        'Winter Weather'
    ]
), 'META_TYPE'] = 'Cold'

detail_data.loc[detail_data.EVENT_TYPE.isin(
    [
        'Dense Smoke',
        'Drought',
        'Excessive Heat',
        'Heat',
        'Volcanic Ash',
        'Wildfire'
    ]
), 'META_TYPE'] = 'Heat'

detail_data.loc[detail_data.EVENT_TYPE.isin(
    [
        'Debris Flow',
        'Dense Fog',
        'Lightning',
        'Marine Dense Fog',
        'Marine Lightning'
    ]
), 'META_TYPE'] = 'Other'

detail_data.loc[detail_data.EVENT_TYPE.isin(
    [
        'Astronomical Low Tide',
        'Coastal Flood',
        'Flash Flood',
        'Flood',
        'Heavy Rain',
        'High Surf',
        'Lakeshore Flood',
        'Rip Current',
        'Seiche',
        'Sneakerwave',
        'Storm Surge/Tide',
        'Tsunami',
        'Waterspout'
    ]
), 'META_TYPE'] = 'Water'

detail_data.loc[detail_data.EVENT_TYPE.isin(
    [
        'Dust Devil',
        'Dust Storm',
        'Funnel Cloud',
        'High Wind',
        'Hurricane',
        'Hurricane (Typhoon)',
        'Marine High Wind',
        'Marine Hurricane/Typhoon',
        'Marine Strong Wind',
        'Marine Thunderstorm Wind',
        'Marine Tropical Depression',
        'Marine Tropical Storm',
        'Strong Wind',
        'Thunderstorm Wind',
        'Tornado',
        'Tropical Depression',
        'Tropical Storm'
    ]
), 'META_TYPE'] = 'Wind'
detail_data.META_TYPE = detail_data.META_TYPE.astype("category")
detail_data['META_CODE'] = detail_data.META_TYPE.cat.codes

In [332]:
detail_data[detail_data.META_TYPE=='Heat'].sample()

,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME,CZ_TIMEZONE,INJURIES_DIRECT,...,DAMAGE_CROPS,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,BEGIN_DATE,END_DATE,EVENT_CODE,META_TYPE,META_CODE
1863,90722.0,544254,OKLAHOMA,40.0,Drought,Z,52,BRYAN,CST-6,0,...,NaN,NaN,NaN,NaN,NaN,2014-09-01,2014-09-30 23:59:00,8,Heat,1


# Fill in missing geographic data
The US Census Gazetteer contains a file which provides Lat/Long for every county in the US. Combined with the FIPs codes, we can give estimated locations for events with no clear starting point (about 1/3 of the dataset - things like "drought" or "Strong Wind" that don't have a clearly defined geographic location).
Dataset from https://www.census.gov/geo/maps-data/data/gazetteer2017.html

In [333]:
county_df = pd.read_csv('data/2017_Gaz_counties_national.txt', sep='\t', engine='python')
county_df['GEOID'] = county_df['GEOID'].apply(lambda x: '{0:0>5}'.format(x))
county_df.set_index("GEOID", inplace=True)

In [334]:
county_df.head()

,USPS,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
GEOID,,,,,,,,,
01001,AL,161526,Autauga County,1539614693,25744269,594.449,9.940,32.532237,-86.646440
01003,AL,161527,Baldwin County,4117605847,1133109409,1589.817,437.496,30.659218,-87.746067
01005,AL,161528,Barbour County,2292144656,50538698,885.002,19.513,31.870253,-85.405104
01007,AL,161529,Bibb County,1612165763,9603798,622.461,3.708,33.015893,-87.127148
01009,AL,161530,Blount County,1670079465,15039864,644.821,5.807,33.977358,-86.566440


### Fill in Lat/Lon for events based on county centroid
Create a new column that combines the State and County FIPS so we can look them up

In [ ]:
detail_data['FIPS'] = detail_data['STATE_FIPS']*1000+detail_data['CZ_FIPS']

Try to match up FIPS ids for events with no location info, and fill in the begin lat/lon with that counties centroid

In [ ]:
# iterate through the rows that are currently missing data and try to 
for i, row in detail_data[detail_data.BEGIN_LAT.isnull()].iterrows():
    try:
        location = county_df.loc['{0:0>5}'.format(int(row.FIPS))]
        detail_data.loc[i,'BEGIN_LAT'] = location['INTPTLAT']
        detail_data.loc[i,'BEGIN_LON'] = location['INTPTLONG']
    except Exception as e:
        pass

### TODO: look at filling in the remaining missing points with bp02oc18.dbx from https://www.weather.gov/gis/ZoneCounty

# Create the matricies

In [393]:
test_set = detail_data[
    ~detail_data.BEGIN_LAT.isnull() &
    ~detail_data.BEGIN_LON.isnull() &
    ~detail_data.END_LAT.isnull() &
    ~detail_data.END_LON.isnull() &
    (detail_data.BEGIN_LAT != detail_data.END_LAT) &
    (detail_data.BEGIN_LON != detail_data.END_LON)
].sample(5)

In [ ]:
m = lil_matrix((5783,2460), dtype=np.int8)

In [408]:
for id, row in test_set.iterrows():
    start = (lat_to_index(row.BEGIN_LAT),lon_to_index(row.BEGIN_LON))
    end = (lat_to_index(row.END_LAT),lon_to_index(row.END_LON))
    print(start,end)
    break


(935, 2089) (921, 2091)


In [ ]:
def get_year_matrix(year):
    start = str(year)+'-01-01'
    end = str(year+1)+'-01-01'
    year_data = detail_data[(detail_data.BEGIN_DATE>start) & (detail_data.BEGIN_DATE<end)].dropna()
    # trim it to just the continental US
    year_data = year_data[
        (year_data.BEGIN_LON > -124.7844079) &
        (year_data.BEGIN_LON < -66.9513812) &
        (year_data.BEGIN_LAT > 24.7433195) &
        (year_data.BEGIN_LAT < 49.3457868)
    ]
    m = lil_matrix((5783,2460), dtype=np.int8)
    for row in year_data.iterrows():
        row = row[1]
        col_id = lat_to_index(row.BEGIN_LAT)
        row_id = lon_to_index(row.BEGIN_LON)
        
        try:
            m[row_id,col_id] = 1
        except:
            print("bad row:",row)
    return m

In [ ]:
def get_year_month_matrix(year, month):
    start = "{:0d}-{:02d}-01".format(year, month)
    if month == 12:
        month = 1
        year += 1
    else:
        month+=1
    end = "{:0d}-{:02d}-01".format(year, month)
    year_data = detail_data[(detail_data.BEGIN_DATE>start) & (detail_data.BEGIN_DATE<end)].dropna()
    # trim it to just the continental US
    year_data = year_data[
        (year_data.BEGIN_LON > -124.7844079) &
        (year_data.BEGIN_LON < -66.9513812) &
        (year_data.BEGIN_LAT > 24.7433195) &
        (year_data.BEGIN_LAT < 49.3457868)
    ]
    m = lil_matrix((5783,2460), dtype=np.int16)
    for row in year_data.iterrows():
        row = row[1]
        col_id = lat_to_index(row.BEGIN_LAT)
        row_id = lon_to_index(row.BEGIN_LON)
        for r in range(row_id-2, row_id+3):
            for c in range(col_id-2, col_id+3):
                try:
                    m[r,c] = 1
                except:
                    print("bad row:",row)
    return m

get_year_month_matrix(2000,7)

In [405]:
def lat_to_index(x):
    return int(x*100-2774)

def lon_to_index(x):
    return int(abs(x)*100-6695)

In [ ]:
%%time
mats = {}
for year in range(1996, 2017):
    mats[str(year)] = {}
    for month in range(1, 13):
#         print(year, month)
        mats[str(year)]["{:02d}".format(month)] = get_year_month_matrix(year, month)

In [ ]:
lat, long = 42.28,-83.74
lat, long = 32.45,-89.65

detail_data[
    (detail_data.BEGIN_LON > long-1) &
    (detail_data.BEGIN_LON < long+1) &
    (detail_data.BEGIN_LAT > lat-1) &
    (detail_data.BEGIN_LAT < lat+1)
]

In [ ]:
mats['1996']['11']

In [ ]:
def get_actual(lat, lon, month, search_range=.25):
    year = 2016
    start = "{:0d}-{:02d}-01".format(year, month)
    if month == 12:
        month = 1
        year += 1
    else:
        month+=1
    end = "{:0d}-{:02d}-01".format(year, month)
    threshold = 1/(search_range*search_range*100*100)/12
    event_count = len(detail_data[
        (detail_data.BEGIN_DATE>start) & 
        (detail_data.BEGIN_DATE<end) &
        (detail_data.BEGIN_LON > lon-search_range) &
        (detail_data.BEGIN_LON < lon+search_range) &
        (detail_data.BEGIN_LAT > lat-search_range) &
        (detail_data.BEGIN_LAT < lat+search_range)
    ])
    seen_per = event_count/(search_range*search_range*3*100*100)
    
    return seen_per>threshold, seen_per/threshold, seen_per


In [ ]:
def search(lat,long,month,search_range=25):
    col_center = lat_to_index(lat)
    row_center = lon_to_index(long)

    event_count = 0
    total_count = 0
    y = {}
    for year in range(1996,2015):
        m = mats[str(year)]["{:02d}".format(month)]
        y[year] = 0

        for col in range(col_center-search_range, col_center+search_range+1):
            for row in range(row_center-search_range, row_center+search_range+1):
                if row >= 0 and row <= 5782 and col >= 0 and col <= 2459:
                    total_count += 1
                    val = m[row,col]
                    if val > 0:
                        event_count += val
                        y[year] += val
                    
    count = 0
    for e in y.values():
        if e > 30:
            count += 1
    prob = min(count/len(d)*4,.99)
    
    return prob > 0.6, prob

In [ ]:
lat,long,month = 42.16,-83.44,7
print("Ann Arbor", search(lat,long,month), get_actual(lat,long,month))

In [ ]:
d = {1996: 0, 1997: 0, 1998: 0, 1999: 0, 2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 219, 2008: 49, 2009: 0, 2010: 285, 2011: 756, 2012: 253, 2013: 139, 2014: 131}

In [ ]:
count = 0
for e in d.values():
    if e > 30:
        count += 1
count/len(d)

In [ ]:
import random
correct = 0
total = 0
pos = 0
for random_sample_row in detail_data[(detail_data.BEGIN_DATE>'2015-01-01') & (detail_data.BEGIN_DATE<'2016-01-01')].dropna().sample(500).iterrows():
    random_sample_row = random_sample_row[1]
    lat,lon = random_sample_row.BEGIN_LAT, random_sample_row.BEGIN_LON
    event_type = random_sample_row.EVENT_TYPE
    month = random.randint(1,12)
    
    prediction = search(lat,lon,month)
    actual = get_actual(lat,lon,month)
    
#     print("({}/{}-{}) {} - Prediction: {}, Actual: {}".format(lat,lon,month,event_type,prediction,actual))
    
    total += 1
    if prediction[0] == actual[0]:
        correct += 1
    if prediction[0]:
        pos += 1

print("total: {:2f}".format(correct/total*100))
print("positives:",pos)

In [ ]:
%%time
import random
correct = 0
total = 0
results = {"pos":0,"neg":0,"false_pos":0,"false_neg":0}
over_99 = {"pos":0,"neg":0}
for random_sample_row in detail_data[(detail_data.BEGIN_DATE>'2015-01-01') & (detail_data.BEGIN_DATE<'2016-01-01')].dropna().sample(500).iterrows():
    random_sample_row = random_sample_row[1]
    lat,lon = random_sample_row.BEGIN_LAT, random_sample_row.BEGIN_LON
    month = random.randint(1,12)
    
    prediction = search(lat,lon,month)
    actual = get_actual(lat,lon,month)
    
    print("({}/{}-{}) Prediction: {}, Actual: {}".format(lat,lon,month,prediction,actual))
    
    total += 1
    if prediction[0] == actual[0]:
        correct += 1
    
    if prediction[0] == True and actual[0] == True:
        results["pos"] += 1
    elif prediction[0] == False and actual[0] == False:
        results["neg"] += 1
    elif prediction[0] == True and actual[0] == False:
        results["false_pos"] += 1
    elif prediction[0] == False and actual[0] == True:
        results["false_neg"] += 1
        
    if prediction[1] == 0.99:
        if actual[0] == True:
            over_99['pos'] += 1
        else:
            over_99['neg'] += 1
        

print("total: {:2f}".format(correct/total*100))
print(results)
print(over_99)

In [ ]:
(122+244)/500
67/500

In [ ]:
lat,long,month = 40.71,-74.00,1
print("Chicago", search(lat,long,month), get_actual(lat,long,month))

lat,long,month = 42.21,71.03,3
print("Boston", search(lat,long,month), get_actual(lat,long,month))

lat,long,month = 42.16,-83.44,7
print("Ann Arbor", search(lat,long,month), get_actual(lat,long,month))


lat,long,month = 29.99,-91.07,2
print("Assumption, LA", search(lat,long,month), get_actual(lat,long,month))

lat,long,month = 34.73,-96.15,5
print("Caddo, OK", search(lat,long,month), get_actual(lat,long,month))

lat,long,month = 29.86,-95.39,5
print("Harris, TX", search(lat,long,month), get_actual(lat,long,month))


lat,long,month = 34.05,-118.25,6
print("Los Angeles", search(lat,long,month), get_actual(lat,long,month))

lat,long,month = 32.42, -117.09,7
print("San Diego", search(lat,long,month), get_actual(lat,long,month))

lat,long,month = 34.16, -118.44,11
print("Simi Valley", search(lat,long,month), get_actual(lat,long,month))

lat,long,month = 38.5, -104.494,7
print("Colorado Springs", search(lat,long,month), get_actual(lat,long,month))

In [ ]:
for month in range(1,13):
    lat,long = 34.05,-118.25
    print("Los Angeles", search(lat,long,month), get_actual(lat,long,month))

# Utility Functions

In [391]:
# from http://www.roguebasin.com/index.php?title=Bresenham%27s_Line_Algorithm
def bresenham_line(start, end):
    """Bresenham's Line Algorithm
    Produces a list of tuples from start and end
 
    >>> points1 = get_line((0, 0), (3, 4))
    >>> points2 = get_line((3, 4), (0, 0))
    >>> assert(set(points1) == set(points2))
    >>> print points1
    [(0, 0), (1, 1), (1, 2), (2, 3), (3, 4)]
    >>> print points2
    [(3, 4), (2, 3), (1, 2), (1, 1), (0, 0)]
    """
    # Setup initial conditions
    x1, y1 = start
    x2, y2 = end
    dx = x2 - x1
    dy = y2 - y1
 
    # Determine how steep the line is
    is_steep = abs(dy) > abs(dx)
 
    # Rotate line
    if is_steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
 
    # Swap start and end points if necessary and store swap state
    swapped = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        swapped = True
 
    # Recalculate differentials
    dx = x2 - x1
    dy = y2 - y1
 
    # Calculate error
    error = int(dx / 2.0)
    ystep = 1 if y1 < y2 else -1
 
    # Iterate over bounding box generating points between start and end
    y = y1
    points = []
    for x in range(x1, x2 + 1):
        coord = (y, x) if is_steep else (x, y)
        points.append(coord)
        error -= abs(dy)
        if error < 0:
            y += ystep
            error += dx
 
    # Reverse the list if the coordinates were swapped
    if swapped:
        points.reverse()
    return points